<a href="https://colab.research.google.com/github/sinungadi/TwitterABSA/blob/master/zero-shot%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 121.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00


In [3]:
import torch
from transformers import pipeline, BertConfig, BertModel

In [4]:
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [5]:
torch.cuda.is_available()

True

In [6]:
data = pd.read_csv("data_filtered.csv")

In [7]:
docs = data['text_cleaned']

In [60]:
CANDIDATE_LABELS = [
    'performa kendaraan',
    'jarak tempuh',
    'infrastruktur pengisian',
    'harga & biaya pemeliharaan',
    'subsidi & insentif',
    'kelestarian lingkungan',
    'ekosistem baterai',
    'desain & tampilan'
]

In [63]:
def getModel():
  device = 0 if torch.cuda.is_available() else 'cpu'
  classifier = pipeline("zero-shot-classification",
                  model = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
                  device = device)
  return classifier

In [20]:
def categorize(data,classifier,multi_label=False,use_fast=True):
  resultData = []

  for text in tqdm(data):
    result = classifier(text, CANDIDATE_LABELS, multi_label = multi_label, use_fast = use_fast)
    resultData.append(result)

  return resultData

In [64]:
classifier = getModel()

In [79]:
test = categorize(docs.sample(50),classifier)

100%|██████████| 50/50 [00:07<00:00,  6.25it/s]


In [80]:
test

[{'sequence': 'Lambat laun mereka have untuk adapt. If kamu think charge battery is a hassle then up untuk you. Majoriti yang saya tgok mengeluh pasal charging waktu ini bukan pakai EV pun Sama macam yang asyik mengeluh pasal RFID asyik sangkut tapi dia pakai smarttag, bukan lane dia yang penuh',
  'labels': ['infrastruktur pengisian',
   'ekosistem baterai',
   'performa kendaraan',
   'subsidi & insentif',
   'jarak tempuh',
   'desain & tampilan',
   'harga & biaya pemeliharaan',
   'kelestarian lingkungan'],
  'scores': [0.4029572308063507,
   0.23947222530841827,
   0.10483655333518982,
   0.10301104187965393,
   0.08480508625507355,
   0.03109886310994625,
   0.02295398712158203,
   0.01086489949375391]},
 {'sequence': 'Ketua Umum Perkumpulan Industri Kendaraan Listrik Indonesia (Periklindo), Moeldoko menyampaikan, "Saya memberikan apresiasi kepada Chandra Asri yang memiliki komitmen kuat untuk terlibat dalam program pemerintah untuk efisiensi dan lingkungan.”',
  'labels': ['kel

In [53]:
def generateTemp():
    return {
        'text' : None,
        'aspect' : None,
        'score' : None
    }

In [ ]:
temp = generateTemp()

temp.update({
    'text' : None,
    'aspect' : None,
    'score' : None
})

In [69]:
test[0]['labels'][:3]

['infrastruktur pengisian', 'kelestarian lingkungan', 'ekosistem baterai']

In [68]:
for i in range(len(CANDIDATE_LABELS)):
  print(CANDIDATE_LABELS[i])

performa kendaraan
jarak tempuh
infrastruktur pengisian
harga & biaya pemeliharaan
subsidi & insentif
kelestarian lingkungan
ekosistem baterai
desain & tampilan


In [78]:
t = test[4]['scores']
max([i for i, x in enumerate(t) if x>=0.9])

1

1